In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity, SEIRHD_Bed

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.5),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_hq': (0, 70),
    'T_recov_non_oxy': (0, 20),
    'T_recov_oxy': (0, 20),
    'T_recov_icu': (0, 20),
    'T_recov_vent': (0, 20),
    'P_non_oxy': (0, 0.15),
    'P_oxy': (0, 0.15),
    'P_icu': (0, 0.05),
    'P_vent': (0, 0.05),
    'P_fatal': (0, 0.2),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, data_from_tracker=True, granular_data=True, filename=None, #Data
    model=SEIRHD_Bed, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
    train_period=14, val_period=0, num_evals=100, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['hospitalised', 'deceased', 'hq', 'non_o2_beds', 'o2_beds', 'icu', 'ventilator'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
# predictions_dict['data_last_date'] = predictions_dict['m2']['data_last_date']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['df_loss']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['best_params']

## Gridsearch + Bayesian Opt

In [ ]:
default_params = predictions_dict[('Maharashtra', 'Mumbai')]['m1']['best_params']
train_period = 7
loss_indices = [-2*train_period, -train_period]

In [ ]:
get_data()

In [ ]:
optimiser = Optimiser()
optimiser.gridsearch(df_true, default_params, variable_param_ranges, model=SEIRHD_Severity, method='mape', 
                     loss_indices=loss_indices, which_compartments=['total_infected'])

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)